In [95]:
#start out with a dataframe
#seperate X then 
import pandas as pd
import numpy as np
from collections import Counter



class node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    
def gini(y):
        #need freq
        classes = np.unique(y)
        impurity = 1.00
        #amount of instances of c divided by the length of our predicted column
        for c in classes:
            p_c = np.sum(c == y)/len(y)
            impurity -= p_c**2
        return impurity


    

        
    

class DecisionTree:
    def __init__(self, max_depth=100, num_feats=None):
        self.max_depth = max_depth
        self.root = None
        self.num_feats = num_feats
    #X feature matrix y target labels depth depth of the tree
    def fit(self, X,y):
        self.num_feats = X.shape[1] if not self.num_feats else min(X.shape[1], self.num_feats)
        root = self.grow_tree(X, y, 0)
        self.root = root
       
    def grow_tree(self, X, y, depth = 0):
        n_samples, n_features = X.shape;
         #only one element in target labels
        if len(np.unique(y)) == 1:
            return node(value = np.unique(y))
        #reached max depth should return the most common element
        if self.max_depth is not None and depth > self.max_depth:
            return node(value = np.bincount(y).argmax())
        #randomly generate features to split on
        feat_idx = np.random.choice(n_features, self.num_feats, replace=False)
        best_threshold, best_feature = self.best_split(X,y,feat_idx)
        if best_feature is None:
            return Node(value=self.most_common_label(y))
        X_left, X_right, y_left, y_right = self.split(X, y, best_feature, best_threshold)
        left = self.grow_tree(X_left, y_left, depth+1)
        right = self.grow_tree(X_right, y_right, depth+1)
        return node(best_feature, best_threshold, left, right)


    def split(self, X, y, feature, threshold):
        left_indices = np.where(X[:, feature] < threshold)[0]
        right_indices = np.where(X[:, feature] >= threshold)[0]
        return X[left_indices], X[right_indices], y[left_indices], y[right_indices]
        

    def best_split(self, X,y, feature_indices):
        best_impurity = np.inf
        
        
        split_idx, split_threshold = None,None
        # how do we pick threshold
        for feature_index in feature_indices:
            X_vals = X[:, feature_index]
            thresholds = np.unique(X_vals)
            
            for threshold in thresholds:
                X_left, X_right, y_left, y_right = self.split(X, y, feature_index, threshold)
                if  len(X_left) == 0 or len(X_right) == 0:
                    continue
                impurity = (len(y_left)/len(y)) * gini(y_left) + gini(y_right) * (len(y_right)/len(y))
                if impurity < best_impurity:
                    split_idx = feature_index
                    split_threshold = threshold
        return split_threshold, split_idx
            
        
    def most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]
        
    def predict(self, X):
        result = []
        for x in X:
            result.append(self.traverse(x, self.root))
        return result
    
    def traverse(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self.traverse(x, node.left)
        else:
            return self.traverse(x, node.right)
            
        

In [120]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

data = datasets.load_iris()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .1)



classifier = DecisionTree()
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
# print(X_test,y_test)
print(predictions)
print(1-mae(y_test,predictions))



[array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([2]), array([1]), array([2]), array([0]), array([0]), array([2]), array([2])]
0.9333333333333333
